In [34]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
from distributed import Client
from dask_jobqueue import SLURMCluster

from IPython.display import display

import pandas as pd
from sklearn.model_selection import train_test_split

In [35]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
# LOCAL = True
LOCAL = False

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='4GB', cores=, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(100)

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40903 instead
  warnings.warn(


Client Scheduler: tcp://10.32.35.120:35739 Dashboard: http://10.32.35.120:40903/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/ratings.csv")
# ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv")
# mov = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/movies.csv")
ratings = pd.DataFrame(ratings.compute())
# mov = pd.DataFrame(mov.compute())

In [ ]:
# mov


In [ ]:
# ratings = ratings.merge(mov, how = 'right', on = "movieId")

In [ ]:
ratings

In [ ]:

user = pd.DataFrame(ratings.userId.unique())
train_user, testval_user = train_test_split(user, test_size=0.4)


movie = pd.DataFrame(ratings.movieId.unique())
temp1_movie, temp2_movie = train_test_split(movie, test_size = 0.7)
train_set = ratings[ratings['userId'].isin(list(train_user[0]))]
testval_set = ratings[ratings['userId'].isin(list(testval_user[0]))]
train_sending = testval_set[testval_set['movieId'].isin(list(temp1_movie[0]))]
testval_cluster =testval_set[testval_set['movieId'].isin(list(temp2_movie[0]))]

temp = pd.DataFrame(testval_set.userId.unique())
val_user, test_user = train_test_split(temp, test_size = 0.5)
val_set = testval_cluster[testval_cluster['userId'].isin(list(val_user[0]))]
test_set = testval_cluster[testval_cluster['userId'].isin(list(test_user[0]))]

In [ ]:
val_set

In [ ]:
test_set

In [ ]:
testval_set

In [ ]:
# concatenate back to train set
final_train = pd.concat([train_set, train_sending])
# ran into concat kernel dead error
# convert it back to dask dataframe
# train_set=  dd.from_pandas(train_set, chunksize = 100000)
# train_sending = dd.from_pandas(train_sending, chunksize= 100000)

In [ ]:
# final_train = dd.concat([train_set, train_sending], axis=0)

In [ ]:
final_train

In [ ]:
train_small_data = final_train
val_small_data = val_set
test_small_data = train_set
# train_large_data = final_train
# val_large_data = val_set
# test_large_data = train_set

In [ ]:
train_small_data.to_csv("train_small_data.csv")
val_small_data.to_csv("val_small_data.csv")
test_small_data.to_csv("test_small_data.csv")
# train_large_data.to_csv("train_large_data.csv")
# val_large_data.to_csv("val_large_data.csv")
# test_large_data.to_csv("test_large_data.csv")